<a href="https://colab.research.google.com/github/nzcbass/Pythontoopenai/blob/main/Final_Candidate_Comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import pandas as pd
from datetime import datetime, timedelta

# DF1 Run Candidate comments from FT 1/9/22 onwards and save down as CandidateComments.csv
# df3 upload Perioddatesweeks
# run filled and closed jobs report save as JobOrders.csv from 1/9/22 onwards
# Run AEWV report from FT and save as AEWVList.csv


# THis code works up to generating next action and next action date
# The next script provides missing activity with dates and future comments from FT


file_path1 = '/content/CandidateComments.csv'
file_path3 = '/content/Perioddatesweeks.csv'
file_path4 = '/content/JobOrders.csv'
file_path5 = '/content/AEWVList.csv'

columns_to_include = ['Entity Name', 'Client Name', 'Contact Name', 'Candidate Name', 'Comment Type', 'Standard Description', 'Comment Date', 'Comment']

df1 = pd.read_csv(file_path1, usecols=columns_to_include)
# df2 = pd.read_csv(file_path2)
df3 = pd.read_csv(file_path3)
df4 = pd.read_csv(file_path4)
df5 = pd.read_csv(file_path5)

# Remove all rows where 'Client Name' is 'Turbo Staff'
df4 = df4[df4['Client Name'] != 'Turbo Staff Ltd']

# Remove the 'Office' column from df4
df4 = df4.drop('Office', axis=1)

# Drop rows where 'Position Type' column includes the word 'Permanent'
df4 = df4[~df4['Position Type'].str.contains('Permanent', na=False)]

# Convert 'Start Date' to datetime with the correct format
df4['Start Date'] = pd.to_datetime(df4['Start Date'], format='%d/%m/%Y')

# Sort the DataFrame by 'Candidate Name' and 'Start Date'
df4 = df4.sort_values(by=['Candidate Name', 'Start Date'])

# Define a function that takes a subset DataFrame and replaces the 'Job Owner' and 'Client Name' values
def update_job_owner_and_client(group):
    if len(group) > 1:  # More than one entry for the candidate
        # Set the 'Job Owner' of the first job to be the 'Job Owner' of the last job
        group['Job Owner'].iloc[0] = group['Job Owner'].iloc[-1]
        # Set the 'Client Name' of the first job to be the 'Client Name' of the last job
        group['Client Name'].iloc[0] = group['Client Name'].iloc[-1]
    return group

# Apply the function to each group of 'Candidate Name'
df4 = df4.groupby('Candidate Name', as_index=False).apply(update_job_owner_and_client)

# Depending on the pandas version, the indexing might be altered, so reset it
df4.reset_index(drop=True, inplace=True)

# Drop duplicates, keeping the first (earliest date) for each candidate
df4 = df4.drop_duplicates(subset='Candidate Name', keep='first')

df4.to_csv('df4.csv', index=False)

# Remove rows where 'Status' column contains the word 'inactive'
df5 = df5[df5['Status'].str.lower() != 'inactive']

# Remove leading whitespaces from 'First Name' and 'Surname'
df5['First Name'] = df5['First Name'].str.strip()
df5['Surname'] = df5['Surname'].str.strip()

# Create new columns 'Full Name' by merging 'First Name' and 'Surname' for DF5
df5['Full Name'] = df5['First Name'] + ' ' + df5['Surname']

# # Merge df4 and df5 to check for matching names
# merged_df = df4.merge(df5, left_on='Candidate Name', right_on='Full Name', how='left')

# # Create the 'AEWV' column based on the merge result
# merged_df['AEWV'] = merged_df.apply(lambda row: 'Yes' if pd.notna(row['Full Name']) else 'No', axis=1)

# # Update the 'AEWV' column in df4 with values from merged_df
# df4['AEWV'] = merged_df['AEWV']

# Create a set of full names from df5 for efficient lookup
full_names_set = set(df5['Full Name'])

# Function to check if a candidate name is in the full names set
def check_name_in_df5(candidate_name):
    return 'Yes' if candidate_name in full_names_set else 'No'

# Apply the function to each row in df4 and create the new 'AEWV' column
df4['AEWV'] = df4['Candidate Name'].apply(check_name_in_df5)

# Remove rows where "entity name" is blank
df1 = df1.dropna(subset=["Entity Name"])

# Rename columns to match output
df1['Standard Description'] = df1['Standard Description'].str.replace('1st Day Call', '1. 1st Day Call')
df1['Standard Description'] = df1['Standard Description'].str.replace('F/up - End of the 1st week', '2. F/up - End of the 1st week')
df1['Standard Description'] = df1['Standard Description'].str.replace('F/up - End of the 2nd week', '3. F/up - End of the 2nd week')
df1['Standard Description'] = df1['Standard Description'].str.replace('F/up - End of the 3rd week', '4. F/up - End of the 3rd week')
df1['Standard Description'] = df1['Standard Description'].str.replace('F/up - End of the 4th week', '5. F/up - End of the 4th week')
df1['Standard Description'] = df1['Standard Description'].str.replace('F/up - End of the 6th week', '6. F/up - End of the 6th week')
df1['Standard Description'] = df1['Standard Description'].str.replace('F/up - End of the 8th week', '7. F/up - End of the 8th week')
df1['Standard Description'] = df1['Standard Description'].str.replace('Month 4 F/up', '9. Month 4 F/up')
df1['Standard Description'] = df1['Standard Description'].str.replace('Month 5 F/up', '10. Month 5 F/up')
df1['Standard Description'] = df1['Standard Description'].str.replace('18. Month 13 F/up Due Date Calculated', '18. Month 13 F/up')
df1['Standard Description'] = df1['Standard Description'].str.replace('19. Month 14 F/up Due Date Calculated', '19. Month 14 F/up')
df1['Standard Description'] = df1['Standard Description'].str.replace('20. Month 15 F/up Due Date Calculated', '20. Month 15 F/up')
df1['Standard Description'] = df1['Standard Description'].str.replace('21. Month 16 F/up Due Date Calculated', '21. Month 16 F/up')
df1['Standard Description'] = df1['Standard Description'].str.replace('22. Month 17 F/up Due Date Calculated', '22. Month 17 F/up')
df1['Standard Description'] = df1['Standard Description'].str.replace('23. Month 18 F/up Due Date Calculated', '23. Month 18 F/up')

merged_df = df4.merge(df1[df1["Comment Type"] == "Candidate"], left_on=["Candidate Name"], right_on=["Entity Name"], how="left")

# Comments to check
comments_to_check = ["1. 1st Day Call", "2. F/up - End of the 1st week", "3. F/up - End of the 2nd week",
                     "4. F/up - End of 3rd week", "5. F/up - End of the 4th week", "6. F/up - End of the 6th week",
                     "7. F/up - End of the 8th week", "8. Month 3 F/up", "9. Month 4 F/up", "10. Month 5 F/up",
                     "11. Month 6 F/up", "12. Month 7 F/up", "13. Month 8 F/up", "14. Month 9 F/up",
                     "15. Month 10 F/up", "16. Month 11 F/up", "17. Month 12 F/up", "18. Month 13 F/up",
                     "19. Month 14 F/up", "20. Month 15 F/up", "21. Month 16 F/up", "22. Month 17 F/up", "23. Month 18 F/up"]

# Create columns for each comment and populate with 'yes' if the comment exists
for comment in comments_to_check:
    merged_df[comment] = merged_df.apply(lambda row: 'Yes' if comment in str(row["Standard Description"]) else '', axis=1)

# Group by 'Candidate Name_x' and aggregate 'yes' values
grouped_df = merged_df.groupby('Candidate Name_x', as_index=False).agg(
    {comment: 'max' for comment in comments_to_check}
)

# Merge grouped_df with df4 to keep all columns
consolidated_df = df4.merge(grouped_df, left_on='Candidate Name', right_on='Candidate Name_x', how='left')

print(consolidated_df)

# Define the due_date_rules dictionary
due_date_rules = {
    '1. 1st Day Call': timedelta(days=0),
    '2. F/up - End of the 1st week': timedelta(days=7),
    '3. F/up - End of the 2nd week': timedelta(days=14),
    '4. F/up - End of 3rd week': timedelta(days=21),
    '5. F/up - End of the 4th week': timedelta(days=28),
    '6. F/up - End of the 6th week': timedelta(days=42),
    '7. F/up - End of the 8th week': timedelta(days=56),
    '8. Month 3 F/up': timedelta(days=84),
    '9. Month 4 F/up': timedelta(days=112),
    '10. Month 5 F/up': timedelta(days=140),
    '11. Month 6 F/up': timedelta(days=168),
    '12. Month 7 F/up': timedelta(days=196),
    '13. Month 8 F/up': timedelta(days=224),
    '14. Month 9 F/up': timedelta(days=252),
    '15. Month 10 F/up': timedelta(days=280),
    '16. Month 11 F/up': timedelta(days=308),
    '17. Month 12 F/up': timedelta(days=336),
    '18. Month 13 F/up': timedelta(days=364),
    '19. Month 14 F/up': timedelta(days=392),
    '20. Month 15 F/up': timedelta(days=420),
    '21. Month 16 F/up': timedelta(days=448),
    '22. Month 17 F/up': timedelta(days=476),
    '23. Month 18 F/up': timedelta(days=494),
    '24. Month 19 F/up': timedelta(days=522),
    '25. Month 20 F/up': timedelta(days=546),
    '26. Month 21 F/up': timedelta(days=574),
    '27. Month 22 F/up': timedelta(days=602),
    '28. Month 23 F/up': timedelta(days=630),
    '29. Month 24 F/up': timedelta(days=658),
    '30. Month 25 F/up': timedelta(days=686)
    # ... Add more columns and due date rules as needed
}

# print(consolidated_df)

# # Save the modified DataFrame to a CSV file
consolidated_df.to_csv('candicommentsjobsmerged.csv', index=False)

# Convert 'Start Date' column to datetime type
consolidated_df['Start Date'] = pd.to_datetime(consolidated_df['Start Date'], format='%d/%m/%Y')

# Iterate over columns and calculate due dates
for column, rule in due_date_rules.items():
    consolidated_df[column + ' Due Date'] = consolidated_df['Start Date'].apply(lambda start_date: start_date + rule)

# Calculate today's date
today = datetime.today().date()

# Create new columns to store next action and next due date
consolidated_df['Next Action'] = ""
consolidated_df['Next Due Date'] = None

# Iterate over rows and find next action and due date
for index, row in consolidated_df.iterrows():
    next_action = None
    next_due_date = None

    for column, rule in due_date_rules.items():
        due_date = row['Start Date'] + rule
        if due_date > today and (next_due_date is None or due_date < next_due_date):
            next_due_date = due_date
            next_action = column

    consolidated_df.at[index, 'Next Action'] = next_action
    consolidated_df.at[index, 'Next Due Date'] = next_due_date

# Convert 'Next Due Date' column to datetime type
consolidated_df['Next Due Date'] = pd.to_datetime(consolidated_df['Next Due Date'], errors='coerce')

# Drop rows with missing or invalid 'Next Due Date' values
consolidated_df = consolidated_df[consolidated_df['Next Due Date'].notna()]


# Change the date format in the "Next Due Date" column to "yyyy/mm/dd"
consolidated_df['Next Due Date'] = pd.to_datetime(consolidated_df['Next Due Date'], format='%d/%m/%y').dt.strftime('%Y/%m/%d')

# Print the modified DataFrame
# print(consolidated_df)

# Save the modified DataFrame to a CSV file
consolidated_df.to_csv('candicommentsjobsmerged.csv', index=False)


<ipython-input-64-bb3d7b6d4450>:52: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df4 = df4.groupby('Candidate Name', as_index=False).apply(update_job_owner_and_client)
<ipython-input-64-bb3d7b6d4450>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['First Name'] = df5['First Name'].str.strip()
<ipython-input-64-bb3d7b6d4450>:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

            Job Owner                      Client Name          Position  \
0     Samaria October                     Test Company         Carpenter   
1     Clayton Badland              CB Civil & Drainage   Civil Carpenter   
2       Thomas Denyer  Fitzgerald Construction Limited         Carpenter   
3    Eugene Lamprecht             VAE NZ Ltd - Waikato       Duct Fitter   
4    Brittany Skelton                   Rokane Limited         Carpenter   
..                ...                              ...               ...   
252    Morsal Darwesh    Caldwell and Levesque Limited       Electrician   
253  Eugene Lamprecht                 Aotea Electrical       Electrician   
254     Thomas Denyer  Fitzgerald Construction Limited         Carpenter   
255      Elliot Dixon       Isaac Construction Dunedin  Machine Operator   
256     Raeleen Evans        VAE NZ Ltd - Christchurch       Duct Fitter   

     Status Report to Contact Start Date    End Date      Candidate Name  \
0    Closed

<ipython-input-64-bb3d7b6d4450>:196: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if due_date > today and (next_due_date is None or due_date < next_due_date):


In [66]:
import pandas as pd
from datetime import datetime

# Load your DataFrame
file_path1 = '/content/candicommentsjobsmerged.csv'
df1 = pd.read_csv(file_path1)

# List of activity columns in numerical order
activity_columns = [
    '1. 1st Day Call', '2. F/up - End of the 1st week', '3. F/up - End of the 2nd week',
    '4. F/up - End of 3rd week', '5. F/up - End of the 4th week', '6. F/up - End of the 6th week',
    '7. F/up - End of the 8th week', '8. Month 3 F/up', '9. Month 4 F/up', '10. Month 5 F/up',
    '11. Month 6 F/up', '12. Month 7 F/up', '13. Month 8 F/up', '14. Month 9 F/up',
    '15. Month 10 F/up', '16. Month 11 F/up', '17. Month 12 F/up', '18. Month 13 F/up',
    '19. Month 14 F/up', '20. Month 15 F/up', '21. Month 16 F/up', '22. Month 17 F/up',
    '23. Month 18 F/up', '24. Month 19 F/up', '25. Month 20 F/up', '26. Month 21 F/up',
    '27. Month 22 F/up', '28. Month 23 F/up', '29. Month 24 F/up', '30. Month 25 F/up' # Add any additional columns here
]

# Find the index of '17. Month 12 F/up'
# insert_index = df1.columns.get_loc('17. Month 12 F/up')

# Add missing columns in reversed numerical order after '17. Month 12 F/up'
missing_columns = sorted(set(activity_columns) - set(df1.columns), key=lambda x: int(x.split('.')[0]), reverse=True)
for col in missing_columns:
    df1.insert(insert_index + 1, col, '')

# Define a dictionary to map activity columns to due date columns
activity_to_due_date = {activity_col: f'{activity_col} Due Date' for activity_col in activity_columns}

# Remove rows where 'AEWV' column is 'No'
df1 = df1[df1['AEWV'] != 'No']

# Remove rows where 'AEWV' column is blank
df1 = df1[df1['AEWV'].notna()]

# Loop through each row
for index, row in df1.iterrows():
    for activity_col, due_date_col in activity_to_due_date.items():
        if activity_col not in row:
            continue  # Skip columns not present in the DataFrame

        if row[activity_col] == 'Yes':
            continue  # If it's 'Yes', move to the next column

        due_date = row.get(due_date_col)  # Use .get() to handle missing columns
        if pd.isna(due_date):
            continue  # If due date is also missing, move to the next column

        # Convert due date to datetime object
        due_date = pd.to_datetime(due_date)

        # Check if the due date is in the past or today
        if due_date <= datetime.today():
            df1.at[index, activity_col] = due_date.strftime('%Y-%m-%d')  # Populate the cell

# List of columns to remove
columns_to_remove = [
    'Status', 'Report to Contact', 'End Date', 'Candidate Name', 'Candidate ID',
    'Job No.', 'Position Type', 'Type', 'Filled Date', 'Parent Name', 'Client No.', 'Cost Centre ID',
    'Cost Centre Name', ' Order by Contact', 'Site Contact', 'Order Date & Time',
    '1. 1st Day Call Due Date', '2. F/up - End of the 1st week Due Date',
    '3. F/up - End of the 2nd week Due Date', '4. F/up - End of 3rd week Due Date',
    '5. F/up - End of the 4th week Due Date', '6. F/up - End of the 6th week Due Date',
    '7. F/up - End of the 8th week Due Date', '8. Month 3 F/up Due Date', '9. Month 4 F/up Due Date',
    '10. Month 5 F/up Due Date', '11. Month 6 F/up Due Date', '12. Month 7 F/up Due Date',
    '13. Month 8 F/up Due Date', '14. Month 9 F/up Due Date', '15. Month 10 F/up Due Date',
    '16. Month 11 F/up Due Date', '17. Month 12 F/up Due Date', '18. Month 13 F/up Due Date',
    '19. Month 14 F/up Due Date', '20. Month 15 F/up Due Date', '21. Month 16 F/up Due Date',
    '22. Month 17 F/up Due Date', '23. Month 18 F/up Due Date', '24. Month 19 F/up Due Date',
    '25. Month 20 F/up Due Date', '26. Month 21 F/up Due Date', '27. Month 22 F/up Due Date',
    '28. Month 23 F/up Due Date', '29. Month 24 F/up Due Date', '30. Month 25 F/up Due Date'
]

# Define a function to extract numbers from a column name
def extract_number(col_name):
    # Split the column name by spaces and take the first part which should contain the number
    num_part = col_name.split()[0]
    # Extract digits from the part
    digits = ''.join(filter(str.isdigit, num_part))
    # Convert to integer if not empty, otherwise return a large number so it sorts last
    return int(digits) if digits else float('inf')

# Create a mapping of column names to the extracted numbers
column_mapping = {col: extract_number(col) for col in df1.columns}

# Sort the columns by the extracted numbers
sorted_columns = sorted(df1.columns, key=lambda col: column_mapping[col])

# Reindex the DataFrame with the sorted column names
df1 = df1[sorted_columns]

# Rename 'Start Date' column to 'Date First Employed'
df1 = df1.rename(columns={'Start Date': 'Date First Employed'})

# Sort df1 by 'Job Owner' first, then by 'Candidate Name_x'
df1 = df1.sort_values(by=['Job Owner', 'Candidate Name_x'])

# Remove specified columns
df1 = df1.drop(columns=columns_to_remove)

df1['Next Due Date'] = pd.to_datetime(df1['Next Due Date'])

# List of columns that should come first, in the order you specified
specified_columns = ['Job Owner', 'Client Name', 'Position', 'Date First Employed', 'AEWV', 'Candidate Name_x', 'Next Action', 'Next Due Date']

# List of all columns that start with a number using a list comprehension
numeric_columns = [col for col in df1.columns if col[0].isdigit()]

# Concatenate the lists, ensuring that the specified columns come first
new_column_order = specified_columns + numeric_columns

# Reindex the DataFrame with the new column order
df1 = df1[new_column_order]

# Save the modified DataFrame to a new file
modified_file_path = '/content/MissingAEWVActivitiesDate.csv'
df1.to_csv(modified_file_path, index=False)
